In [2]:
# ------------------------------------------------------------------------------ #
# @Author:        F. Paul Spitzner
# @Email:         paul.spitzner@ds.mpg.de
# @Created:       2023-10-09 16:50:01
# @Last Modified: 2023-10-09 16:50:04
# ------------------------------------------------------------------------------ #
# small demo of how we combined the two separate dataframes that we got from
# `analyse_all_units`, where one contained blocks 6, 8, and the other
# spontaneous activity
# Note that storing of spike-times in the dataframe via hdf5 only works
# when spikes are stored in flat numpy arrays, but not as xarrays
# -> use `utl.load_spikes(meta_df, format="numpy")`
#
# These dataframes still need some cleaning, e.g. for cases where the estimation
# failed.
# ------------------------------------------------------------------------------ #

%reload_ext autoreload
%autoreload 2
%reload_ext ipy_dict_hierarchy
%reload_ext watermark

import logging
logging.basicConfig(
    format="%(asctime)s | %(levelname)-8s | %(name)-s | %(funcName)-s | %(message)s",
    level=logging.WARNING,
)
log = logging.getLogger("notebook")
log.setLevel("DEBUG")

import sys
import os
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm


# also needs to be added for each dask-worker
extra_path = os.path.abspath('../')
sys.path.append(extra_path)
log.info(f"project directory: {extra_path}")

from ana import utility as utl
from ana import plot_helper as ph

ph.log.setLevel("DEBUG")
utl.log.setLevel("DEBUG")

data_dir = os.path.abspath("/path/to/repo/experiment_analysis/dat/")


2024-07-05 11:28:31,288 | INFO     | notebook | <module> | project directory: /Users/paul/para/5_Archive/information_timescales/repo/_latest/experiment_analysis


In [4]:
frame_1 = pd.read_hdf(f"{data_dir}/all_units_merged_blocks.h5", key="meta_df")
frame_1.set_index(['unit_id', 'stimulus', 'session', 'block'], inplace=True)

frame_2 = pd.read_hdf(f"{data_dir}/all_units_spontaneous_for_merged.h5", key="meta_df")
frame_2.set_index(['unit_id', 'stimulus', 'session', 'block'], inplace=True)

frame_target = pd.concat([frame_1, frame_2], axis=0)
frame_target.to_hdf(f"{data_dir}/all_units_merged_blocks_with_spont.h5", key="meta_df")

In [11]:
%watermark -v --iversions

Python implementation: CPython
Python version       : 3.11.4
IPython version      : 8.14.0

numpy         : 1.24.4
logging       : 0.5.1.2
pandas        : 2.0.3
matplotlib    : 3.7.2
re            : 2.2.1
sys           : 3.11.4 | packaged by conda-forge | (main, Jun 10 2023, 18:08:41) [Clang 15.0.7 ]
h5py          : 3.9.0
sqlite3       : 2.6.0
IPython       : 8.14.0
prompt_toolkit: 3.0.39
xarray        : 2023.7.0
dask          : 2023.7.1

